In [26]:
import pandas as pd
import math as m
import pprint
from neo4j import GraphDatabase
import numpy as np

# Umieść pliki csv w folderze "games" lub zmień ścieżki do plików

In [28]:
games = pd.read_csv('mecze/games.csv', encoding = "ISO-8859-1")[['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID', 'homeGoals', 'awayGoals']]
games.head()

,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3


In [29]:
teams = pd.read_csv('mecze/teams.csv', encoding = "ISO-8859-1")
teams.head()

,teamID,name
0,71,Aston Villa
1,72,Everton
2,74,Southampton
3,75,Leicester
4,76,West Bromwich Albion


In [30]:
players = pd.read_csv('mecze/players.csv', encoding = "ISO-8859-1")
players.head()

,playerID,name
0,560,Sergio Romero
1,557,Matteo Darmian
2,548,Daley Blind
3,628,Chris Smalling
4,1006,Luke Shaw


In [31]:
players['lastName']=players['name']

In [32]:
for i in range(len(players['playerID'])):
    name = players['name'][i].split()[0]
    try:
        lastName = players['name'][i].split()[1]
    except:
        lastName = ''
    players['name'][i]=name
    players['lastName'][i]=lastName

C:\Users\Karolina\AppData\Local\Temp\ipykernel_32156\2575302664.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['name'][i]=name
C:\Users\Karolina\AppData\Local\Temp\ipykernel_32156\2575302664.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players['lastName'][i]=lastName


In [33]:
players.head()

,playerID,name,lastName
0,560,Sergio,Romero
1,557,Matteo,Darmian
2,548,Daley,Blind
3,628,Chris,Smalling
4,1006,Luke,Shaw


In [34]:
leagues = pd.read_csv('mecze/leagues.csv', encoding = "ISO-8859-1")
leagues.head()

,leagueID,name,understatNotation
0,1,Premier League,EPL
1,2,Serie A,Serie_A
2,3,Bundesliga,Bundesliga
3,4,La Liga,La_liga
4,5,Ligue 1,Ligue_1


In [35]:
shots = pd.read_csv('mecze/shots.csv', encoding = "ISO-8859-1")[['gameID','shooterID','minute','shotType','shotResult']]
shots.head()

,gameID,shooterID,minute,shotType,shotResult
0,81,554,27,LeftFoot,BlockedShot
1,81,555,27,RightFoot,BlockedShot
2,81,554,35,LeftFoot,BlockedShot
3,81,554,35,LeftFoot,MissedShots
4,81,555,40,RightFoot,BlockedShot


# Tu należy zmienić hasło do bazy

In [36]:
neo = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth=('neo4j', 'games'))
neo

In [37]:
session=neo.session()
session

In [38]:
query='MATCH (x) RETURN x'

In [39]:
nodes = session.run(query)
nodes

In [40]:
for i in range(len(games['gameID'])):
    create = 'create (g:game{gameID:"' +str(games['gameID'][i]) + '", leagueID:"' +str(games['leagueID'][i]) + '", season:"' +str(games['season'][i]) + '", date:"' +str(games['date'][i]) + '", homeTeamID:"' +str(games['homeTeamID'][i]) + '", awayTeamID:"' +str(games['awayTeamID'][i]) + '", homeGoals:"' +str(games['homeGoals'][i]) + '", awayGoals:"' +str(games['awayGoals'][i]) + '"})'
    session.run(create)

In [41]:
for i in range(len(teams['teamID'])):
    create = 'create (t:team{teamID:"' +str(teams['teamID'][i]) + '", name:"' +str(teams['name'][i]) + '"})'
    session.run(create)

In [42]:
for i in range(len(players['playerID'])):
    create = 'create (p:player{playerID:"' +str(players['playerID'][i]) + '", name:"' +str(players['name'][i]) + '", lastName:"' +str(players['lastName'][i]) + '"})'
    session.run(create)

In [43]:
for i in range(len(leagues['leagueID'])):
    create = 'create (l:league{leagueID:"' +str(leagues['leagueID'][i]) + '", name:"' +str(leagues['name'][i]) + '", understatNotation:"' +str(leagues['understatNotation'][i]) + '"})'
    session.run(create)

In [44]:
for i in range(len(shots['gameID'][:10000])):
    create = 'create (s:shot{gameID:"' +str(shots['gameID'][i]) + '", shooterID:"' +str(shots['shooterID'][i]) + '", minute:"' +str(shots['minute'][i]) + '", shotType:"' +str(shots['shotType'][i]) + '", shotResult:"' +str(shots['shotResult'][i]) + '"})'
    session.run(create)

In [45]:
merge = 'MATCH (g:game) MATCH (l:league) WHERE g.leagueID = l.leagueID MERGE (g)-[:IS_IN_LEAGUE]->(l)'
session.run(merge)

In [46]:
merge = 'MATCH (g:game) MATCH (t:team) WHERE g.homeTeamID = t.teamID MERGE (t)-[:IS_HOME_TEAM]->(g)'
session.run(merge)

In [47]:
merge = 'MATCH (g:game) MATCH (t:team) WHERE g.awayTeamID = t.teamID MERGE (t)-[:IS_AWAY_TEAM]->(g)'
session.run(merge)

In [48]:
merge = 'MATCH (g:game) MATCH (s:shot) WHERE g.gameID = s.gameID MERGE (s)-[:IS_FROM_GAME]->(g)'
session.run(merge)

In [49]:
merge = 'MATCH (p:player) MATCH (s:shot) WHERE p.playerID = s.shooterID MERGE (s)-[:WAS_SHOT_BY]->(p)'
session.run(merge)

In [25]:
appearances = pd.read_csv('mecze/appearances.csv', encoding = "ISO-8859-1")[['gameID', 'playerID', 'goals', 'shots', 'position', 'yellowCard', 'redCard', 'leagueID']]
appearances.head()

,gameID,playerID,goals,shots,position,yellowCard,redCard,leagueID
0,81,560,0,0,GK,0,0,1
1,81,557,0,0,DR,0,0,1
2,81,548,0,0,DC,0,0,1
3,81,628,0,0,DC,0,0,1
4,81,1006,0,0,DL,0,0,1


In [50]:
for i in range(len(appearances['gameID'][:10000])):
    create = 'create (a:appearance{gameID:"' +str(appearances['gameID'][i]) + '", playerID:"' +str(appearances['playerID'][i]) + '", goals:' + str(appearances['goals'][i]) + ', shots:' + str(appearances['shots'][i]) + ', position:"' +str(appearances['position'][i]) + '", yellowCard:' +str(appearances['yellowCard'][i]) + ', redCard:' +str(appearances['redCard'][i]) + ', leagueID:"' +str(appearances['leagueID'][i]) + '"})'
    session.run(create)

In [51]:
merge = 'MATCH (g:game) MATCH (a:appearance) WHERE g.gameID = a.gameID MERGE (g)-[:WAS_PLAYED_BY]->(a)'
session.run(merge)

In [52]:
merge = 'MATCH (p:player) MATCH (a:appearance) WHERE p.playerID = a.playerID MERGE (a)-[:IS_ABOUT]->(p)'
session.run(merge)

In [53]:
merge = 'MATCH (l:league) MATCH (a:appearance) WHERE l.leagueID = a.leagueID MERGE (a)-[:IS_IN_LEAGUE]->(l)'
session.run(merge)

In [57]:
query1 = 'MATCH (t:team{name:"Everton"})-[:IS_HOME_TEAM]->(g:game)<-[:IS_FROM_GAME]-(s:shot{shotResult:"Goal"}) RETURN s UNION MATCH (t:team{name:"Everton"})-[:IS_AWAY_TEAM]->(g:game)<-[:IS_FROM_GAME]-(s:shot{shotResult:"Goal"}) RETURN s'
session.run(query1)
query2 = 'MERGE (s:shot{gameID:"81", shooterID:"554", minute:"29"}) ON CREATE SET s.shotType="LeftFoot", s.shotResult="MissedShots" MERGE (s)-[:IS_FROM_GAME]->(g:game{gameID:"81"}) ON MATCH SET s.shotType="RightFoot", s.shotResult="MissedShots" RETURN s'
query3 = 'MERGE (a:appearance{gameID:"81", playerID:"554"}) ON CREATE SET a.goals=1, a.shots=1, a.position="GK", a.yellowCard=0, a.redCard=0, a.leagueID="1" MERGE (g:game{gameID:"81"})-[:WAS_PLAYED_BY]->(a) MERGE (a)-[:IS_ABOUT]->(p:player{playerID:"554"}) MERGE (a)-[:IS_IN_LEAGUE]->(l:league{leagueID:"1"}) ON MATCH SET a.goals=1, a.shots=2, a.yellowCard=1 RETURN a'
query4 = 'MATCH (g:game{season:"2015"})-[:WAS_PLAYED_BY]->(a:appearance{redCard:1})-[:IS_ABOUT]->(p:player) RETURN p UNION MATCH (g:game{season:"2016"})-[:WAS_PLAYED_BY]->(a:appearance{redCard:1})-[:IS_ABOUT]->(p:player) RETURN p'

In [58]:
index1 = 'CREATE INDEX team_name_index FOR (n:team) ON (n.name)'
index2 = 'CREATE INDEX homeGoal_count_index FOR (n:game) ON (n.homeGoals)'
index3 = 'CREATE INDEX awayGoal_count_index FOR (n:game) ON (n.awayGoals)'
index4 = 'CREATE INDEX player_name_index FOR (n:player) ON (n.name)'
index5 = 'CREATE INDEX player_lastName_index FOR (n:player) ON (n.lastName)'
index6 = 'CREATE INDEX shotResult_index FOR (n:shot) ON (n.shotResult)'
session.run(index1)
session.run(index2)
session.run(index3)
session.run(index4)
session.run(index5)
session.run(index6)

In [62]:
procedure = str("CALL apoc.custom.asProcedure('yellowCards','MATCH (g:game{season:$input3})-[:WAS_PLAYED_BY]->(a:appearance)-[:IS_ABOUT]->(p:player{name:$input1, lastName:$input2}) WITH p, sum(a.yellowCard) as yellowCards RETURN $input1, $input2, yellowCards','read', [['yellowCards', 'INTEGER']], [['input1', 'STRING'],['input2', 'STRING'], ['input3', 'STRING']], 'Returns number of yellow cards given to player in given season.');")
session.run(procedure)